##### Imports

In [1]:
import asvFormula.bayesianNetworks.bayesianNetwork as bn
import asvFormula.experiments as exp
from asvFormula import asvRunResultsPath
from asvFormula.datasetManipulation import *
import asvFormula.ASV as exact
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

cancerApproximateASVFile = asvRunResultsPath + "/cancerASVAndShapleyApproximate.csv"
childApproximateASVFile = asvRunResultsPath + "/childASVAndShapleyApproximate.csv"

cancerExactASVFile = asvRunResultsPath + "/cancerASVAndShapleyExact.csv"
childExactASVFile = asvRunResultsPath + "/childASVAndShapleyExact.csv"

seeds = list(range(100, 125)) + [12,32,53,420,5]

In [2]:
from importlib import reload 
import asvFormula
import asvFormula.bayesianNetworks.bayesianNetwork as bn
import asvFormula.ASV as exact
import asvFormula.experiments as exp

bn = reload(asvFormula.bayesianNetworks.bayesianNetwork)
exact = reload(asvFormula.ASV)
exp = reload(asvFormula.experiments)


## Generating the results 

### Running ASV for Cancer bayesian network

In [3]:
if os.path.exists(cancerApproximateASVFile):
    os.remove(cancerApproximateASVFile) #Remove the file to reset results
for seed in seeds:
    cancerBNmodel, variableToPredict, numberOfSamples, treeMaxDepth = exp.cancerNetworkConfig()

    BNInference, valuesPerFeature, encodedDataset, dtTreeClassifier, dtAsNetwork = initializeDataAndRemoveVariable(cancerBNmodel, variableToPredict, numberOfSamples, treeMaxDepth, seed)

    dataNoPrediction = encodedDataset.drop(variableToPredict, axis=1)
    first_instance = dataNoPrediction.iloc[0]

    asvCalc = exact.ApproximateASV(cancerBNmodel, dtTreeClassifier, BNInference, valuesPerFeature, variableToPredict, predictionFunction='Exact')

    exp.writeASVAndShapleyIntoFile(first_instance, dataNoPrediction, dtTreeClassifier, asvCalc, cancerApproximateASVFile , valuesPerFeature, variableToPredict, seed)

  0%|          | 0/5 [00:00<?, ?it/s]

The model accuracy is : 0.6833333333333333


  0%|          | 0/5 [00:00<?, ?it/s]

The model accuracy is : 0.775


  0%|          | 0/5 [00:00<?, ?it/s]

...

The model accuracy is : 0.7583333333333333


  0%|          | 0/5 [00:00<?, ?it/s]

The model accuracy is : 0.725


  0%|          | 0/5 [00:00<?, ?it/s]

The model accuracy is : 0.7583333333333333


### Running ASV for Child bayesian network

In [4]:
if os.path.exists(childApproximateASVFile):
    os.remove(childApproximateASVFile) #Remove the file to reset results
for seed in seeds:
    treeBNChild, variableToPredict, numberOfSamples, treeMaxDepth = exp.childNetworkConfig()
    BNInference, valuesPerFeature, encodedDataset, dtTreeClassifier, dtAsNetwork = initializeDataAndRemoveVariable(treeBNChild, variableToPredict, numberOfSamples, treeMaxDepth, seed)

    dataNoPrediction = encodedDataset.drop(variableToPredict, axis=1)
    first_instance = dataNoPrediction.iloc[0]

    asvCalc = exact.ApproximateASV(treeBNChild, dtTreeClassifier, BNInference, valuesPerFeature, variableToPredict, predictionFunction='Exact')

    exp.writeASVAndShapleyIntoFile(first_instance, dataNoPrediction, dtTreeClassifier, asvCalc, childApproximateASVFile , valuesPerFeature, variableToPredict, seed)
    #This takes 3 minutes to run per seed

  0%|          | 0/20 [00:00<?, ?it/s]

The model accuracy is : 0.666


  0%|          | 0/20 [00:00<?, ?it/s]

The model accuracy is : 0.6635


  0%|          | 0/20 [00:00<?, ?it/s]

...

The model accuracy is : 0.6465


  0%|          | 0/20 [00:00<?, ?it/s]

The model accuracy is : 0.6785


  0%|          | 0/20 [00:00<?, ?it/s]

The model accuracy is : 0.675


## Plotting the results 

### Obtaining results + converting to numerical data

In [ ]:
dfCancerApproximateDict = exp.multipleSeedsDataframes(cancerApproximateASVFile, seeds)
dfCancerExactDict = exp.multipleSeedsDataframes(cancerExactASVFile, seeds)

dfChildAppproximateDict = exp.multipleSeedsDataframes(childApproximateASVFile, seeds)
dfChildExactDict = exp.multipleSeedsDataframes(childExactASVFile, seeds)

plotsResultsFolder = 'results/plots/'

def calculateASVDifferences(dfExactDict: dict[int, pd.DataFrame], 
                            dfApproximateDict: dict[int, pd.DataFrame], 
                            seeds):

    all_merged = []

    for seed in seeds:
        exact_df = dfExactDict[seed]
        approx_df = dfApproximateDict[seed]

        predictedColumn = "Smoker" if 'cancer' in exact_df.name else "Age"
        merged = pd.merge(exact_df, approx_df, on=["Feature", f"{predictedColumn} value"], 
                          suffixes=("_exact", "_approx"))

        merged = merged[merged['ASV_approx'] > 0.01]


        merged["ASV_diff"] = (merged["ASV_exact"] - merged["ASV_approx"])

        merged["relative_difference"] = np.where(
            merged["ASV_exact"] != 0,
            (merged["ASV_diff"] / merged["ASV_exact"].abs()) * 100,
            np.nan
        )

        merged["Seed"] = seed
        if seed != 101: # Exclude seed 101 as it had some randomness in the results
            all_merged.append(merged)

    # Concatenamos todos los merged en un único dataframe
    final_df = pd.concat(all_merged, ignore_index=True)
    return final_df

### Plotting ASV values for each feature

In [ ]:
def plotASVandShapForSeeds(seeds, dfDict, hueValue):
    _, axes = plt.subplots(len(seeds), 2, figsize=(20, 15))
    for i, seed in enumerate(seeds):
        df = dfDict[seed]
        exp.plotASVandShapFromDF(df, hueValue, seed, None, axes[i])

    bayesianNetwork = 'cancer' if "Smoker value" == hueValue else 'child'
    plt.tight_layout()  # Adjust layout to prevent overlap
    plt.savefig(f"{plotsResultsFolder}{bayesianNetwork}MultipleSeedsASVandShapleyApproximate.png")
    plt.show()

exp.plotASVandShapFromDF(dfCancerApproximateDict[12], 'Smoker value', 12, f"{plotsResultsFolder}{dfCancerApproximateDict[12].name}ASVAndShapley.png")
exp.plotASVandShapFromDF(dfChildAppproximateDict[420], 'Age value', 420, f"{plotsResultsFolder}{dfChildAppproximateDict[420].name}ASVAndShapley.png")

plotASVandShapForSeeds(seeds, dfCancerApproximateDict, 'Smoker value')
plotASVandShapForSeeds(seeds, dfChildAppproximateDict, 'Age value')


### Plotting Exact vs Approximate Differences

In [16]:
# Plot global con todas las seeds

def plot_asv_relative_difference_boxplot(final_df, predictedColumn, plotsResultsFolder):
    plt.figure(figsize=(12, 8))
    sns.boxplot(
        data=final_df,
        x=f"{predictedColumn} value",
        y="relative_difference"
    )

    plt.axhline(0, color="black", linewidth=1)
    plt.title(f"Diferencia Relativa de ASV por Valor de {predictedColumn} (Todas las seeds)")
    plt.ylabel("Diferencia Relativa de ASV")
    plt.xlabel(f"Valor de {predictedColumn}")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f"{plotsResultsFolder}ChildAllSeedsASVBoxplot.png")
    plt.show()

#calculateASVDifferences(dfCancerApproximateDict, dfCancerExactDict, seeds)
difference_df = calculateASVDifferences(dfChildAppproximateDict, dfChildExactDict, seeds)
predictedColumn = "Age"
plot_asv_relative_difference_boxplot(difference_df, predictedColumn, plotsResultsFolder)


KeyError: 'Seed'

### Analyzing Exact vs Approximate Differences

#### Outliers Exact vs Approximate Differences

In [14]:
def displayOutliers(dfExactDict: dict[int, pd.DataFrame], 
                    dfApproximateDict: dict[int, pd.DataFrame], 
                    seeds):

    final_df = calculateASVDifferences(dfExactDict, dfApproximateDict, seeds)

    # Filter rows where Shapley_exact and Shapley_approx are different
    
    filtered = final_df[abs(final_df["relative_difference"]) > 20]
    
    seedsWithProblem = set(filtered["Seed"].unique())
    #print("Number of seeds which have this problem: " + str(seedsWithProblem))
    display(filtered)

#calculateASVDifferences(dfCancerApproximateDict, dfCancerExactDict, seeds)
displayOutliers(dfChildAppproximateDict, dfChildExactDict, seeds)

,Feature,Age value,ASV_exact,Shapley_exact,Seed_exact,ASV_approx,Shapley_approx,Seed_approx,ASV_diff,relative_difference,Seed
8,LVHreport,0-3_days,0.017996,-0.01152,101,0.023102,-0.01152,101,-0.005106,-28.37408,101


#### Datasets for same seed and different kernel

In [ ]:
seed = 100

treeBNChild, variableToPredict, numberOfSamples, treeMaxDepth = exp.childNetworkConfig()

os.environ.setdefault('PYTHONHASHSEED', '0') #THIS SOLVED THE ISSUE
dataFromBN = treeBNChild.simulate(n_samples=numberOfSamples, seed = seed)
dataFromBN = dataFromBN[sorted(dataFromBN.columns)]

dataFromBN.to_csv("testingDataSet2.csv", index=False)

#d1 = pd.read_csv("testingDataSet.csv")
#d2 = pd.read_csv("testingDataSet2.csv")
#d1.equals(d2)


  0%|          | 0/20 [00:00<?, ?it/s]

True

#### Shapley and ASV values for same seed and dataset

In [ ]:
seed = 100
treeBNChild, variableToPredict, numberOfSamples, treeMaxDepth = exp.childNetworkConfig()
BNInference, valuesPerFeature, encodedDataset, dtTreeClassifier, dtAsNetwork = initializeDataAndRemoveVariable(treeBNChild, variableToPredict, numberOfSamples, treeMaxDepth, seed)

dataNoPrediction = encodedDataset.drop(variableToPredict, axis=1)
first_instance = dataNoPrediction.iloc[0]

asvCalc = exact.ApproximateASV(treeBNChild, dtTreeClassifier, BNInference, valuesPerFeature, variableToPredict, predictionFunction='Exact')

childApproximateASVTestingFile = asvRunResultsPath + "/childASVAndShapleyApproximateTesting.csv"

dataSet = dataNoPrediction.copy()
progress = False

  0%|          | 0/20 [00:00<?, ?it/s]

The model accuracy is : 0.6675


In [ ]:
import shap

features = ['Disease'] #, 'ChestXray', 'CO2']
explainer = shap.TreeExplainer(dtTreeClassifier, dataSet)
shap_values = explainer.shap_values(first_instance)
predictionValues = valuesPerFeature[variableToPredict]

# Check if the file exists to decide whether to write the header
file_exists = os.path.isfile(childApproximateASVTestingFile)

with open(childApproximateASVTestingFile, 'a') as f:
    if not file_exists:
        f.write(f"Feature,{variableToPredict} value,ASV,Shapley,Seed\n")

    for i, feature in enumerate(features):
        asvValue = asvCalc.asvForFeature(feature, first_instance, showProgress=progress)
        shapleyValue = shap_values[i]
        for j in range(len(shapleyValue)):
            f.write(f"{feature},{predictionValues[j]},{asvValue[j]},{shapleyValue[j]},{seed}\n")

## Child toposorts generation experiment

In [ ]:
treeBNChild, variableToPredict, numberOfSamples, treeMaxDepth = exp.childNetworkConfig()

In [ ]:
import timeit
from itertools import islice
from asvFormula.topoSorts import allPolyTopoSorts


numberOfSamples = 100
#allToposorts = allPolyTopoSorts(treeBNChild)
timeForSamples = timeit.timeit(lambda: exact.randomTopoSorts(treeBNChild, numberOfSamples), number=1)
#timeForClasses = timeit.timeit(lambda: exact.equivalenceClassesFor(treeBNChild, 'HypDistrib'), number=1)
#timeForExact = timeit.timeit(lambda:  list(islice(nx.all_topological_sorts(treeBNChild), numberOfSamples)), number=1)


#print(allToposorts)
print(f"Time for {numberOfSamples} samples: {timeForSamples}, which is a {round(numberOfSamples/allToposorts*100, 6)}% of total toposorts")
#print(f"Time taken for generating equivalence classes classes : {timeForClasses}")
#print(f"Time for {numberOfSamples} exact generation of toposorts: {timeForExact}, which is a {round(numberOfSamples/allToposorts*100, 6)}% of total toposorts")



Time for 100 samples: 1.226576216000467, which is a 0.0% of total toposorts
